In [13]:
from scipy import stats
import os
import sys
import json
import numpy as np

In [15]:
model = 'mistral-large-2411'
results_p1 = []
results_p2 = []
for prompt in ['prompt_1', 'prompt_2']:
    for alpha in [1, 3, 10]:
        data_path = f'mistral_ai/results/duopoly/{prompt}-alpha_{alpha}'
        folders = [f for f in os.listdir(data_path) if f.endswith(model)]
        results = []
        for folder in folders:
            results_file = os.path.join(data_path, folder, 'results.json')
            with open(results_file, 'r') as f:
                r = json.load(f)
            for firm in ['firm_1', 'firm_2']:
                mean_price = np.mean(r['price_history'][firm][-50:])
                divisor = 3.2 if alpha == 3 else alpha
                if prompt == 'prompt_1':
                    results_p1.append(mean_price/divisor)
                else:
                    results_p2.append(mean_price/divisor)

In [19]:
normality_p1 = stats.shapiro(results_p1)
#interpret normality_p1
normality_p2 = stats.shapiro(results_p2)
#interpret normality_p2
print(f'Normality test for prompt 1: {normality_p1}')
print(f'Normality test for prompt 2: {normality_p2}')
# Perform t-test
t_stat, p_value = stats.ttest_ind(results_p1, results_p2, equal_var=False)
# Interpret the t-test result
if p_value < 0.05:
    print(f'Statistically significant difference between prompt 1 and prompt 2 (p-value: {p_value})')
else:
    print(f'No statistically significant difference between prompt 1 and prompt 2 (p-value: {p_value})')
# Print the means for both prompts
print(f'Mean price for prompt 1: {np.mean(results_p1)}')
print(f'Mean price for prompt 2: {np.mean(results_p2)}')

Normality test for prompt 1: ShapiroResult(statistic=np.float64(0.9044446156769425), pvalue=np.float64(0.006023582515186334))
Normality test for prompt 2: ShapiroResult(statistic=np.float64(0.8585485617455029), pvalue=np.float64(0.0031634399919519725))
Statistically significant difference between prompt 1 and prompt 2 (p-value: 5.718624838168215e-05)
Mean price for prompt 1: 1.8932850147058824
Mean price for prompt 2: 1.60325484375
